<a href="https://colab.research.google.com/github/Riky2014/Tesi/blob/main/1d_hemo_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!apt-get install software-properties-common
!add-apt-repository -y ppa:fenics-packages/fenics
!apt-get update -qq
!apt install fenics

In [2]:
from fenics import *
import numpy as np
import matplotlib.pyplot as plt

In [64]:
# Exact solution and forcing term
A_tilde = 4e-4
a_tilde = 4e-5
q_tilde = 0.0
K_tilde = 50e3

k_r = 2.416e-4
rho = 1.05

T = 1
L = 1

A_exact = Expression('A_tilde + a_tilde * sin(2 * pi / L * x[0]) * cos(2 * pi / T * t)', A_tilde = A_tilde, a_tilde = a_tilde, L = L, T = T, degree = 2, t = 0)
q_exact = Expression('q_tilde - (a_tilde * L / T) * cos(2 * pi / L * x[0]) * sin(2 * pi / T * t)', q_tilde = q_tilde, a_tilde = a_tilde, L = L, T = T, degree = 2, t = 0)

dU_dt = Expression(('- a_tilde * 2 * pi * sin(2 * pi * t) * sin(2 * pi * x[0])',
                    '- a_tilde * L / T * 2 * pi * cos(2 * pi * t) * cos(2 * pi * x[0])'),
                   a_tilde = a_tilde, L = L, T = T, degree = 2, t = 0)

S = Expression(('0.',
                'k_r * q_tilde - (a_tilde * L / T) * cos(2 * pi / L * x[0]) * sin(2 * pi / T * t) / (A_tilde + a_tilde * sin(2 * pi / L * x[0]) * cos(2 * pi / T * t))'),
                A_tilde = A_tilde, a_tilde = a_tilde, q_tilde = q_tilde, L = L, T = T, k_r = k_r, degree = 2, t = 0)

dF_dx = Expression(('2 * pi * a_tilde * L / T * sin(2 * pi * t) * sin(2 * pi * x[0])',
                    'pi * K_tilde * a_tilde / rho * pow(0.1 * sin(2 * pi * x[0]) * cos(2 * pi * t) + 1, 0.5) * cos(2 * pi * t) * cos(2 * pi * x[0]) - 2 * pi * pow(a_tilde, 3) * pow(L / (A_tilde * T), 2) * pow(sin(2 * pi * t), 2) * cos(2 * pi * t) * pow(cos(2 * pi * x[0]), 3) / pow(0.1 * sin(2 * pi * x[0] * cos(2 * pi * t) + 1), 2) - 4 * pi * pow(a_tilde, 2) * pow(sin(2 * pi * t), 2) * sin(2 * pi * x[0]) * cos(2 * pi * x[0]) / (A_tilde + a_tilde * sin(2 * pi * x[0]) * cos(2 * pi * t))'),
                    a_tilde = a_tilde, A_tilde = A_tilde, K_tilde = K_tilde, L = L, T = T, rho = rho, degree = 2, t = 0)

In [172]:
def update_time_step(t, dt):
  t += dt
  q_exact.t = t
  A_exact.t = t
  dU_dt.t = t
  S.t = t
  dF_dx.t = t

In [170]:
f = dU_dt + S + dF_dx

In [173]:
# Data
T = 1
x_left = 0.
x_right = 1.0
alpha = 1

# Discretization parameter
dt = 2e-4
h = 1e-1
num_steps = T / dt
N = int((x_right - x_left) / h)

#  Create a mesh on the interval [0, 1].
mesh = IntervalMesh(N, x_left, x_right)
x = MeshCoordinates(mesh)

# Define the function space
P1 = FiniteElement('P', mesh.ufl_cell(), 1)
element = MixedElement([P1, P1])
V = FunctionSpace(mesh, element)

# Derivative function spoace
D1 = FiniteElement('DG', mesh.ufl_cell(), 0)
element_der = MixedElement([D1, D1])
V_der = FunctionSpace(mesh, element_der)

In [103]:
def H(A, q):
  return as_tensor([[0, 1], [1190476.19048 * A ** 0.5 - (q / A) ** 2, 2 * q / A]])

def F(A, q):
  return as_vector([q, 793650.79365 * A ** 1.5 - 6.34921 + q ** 2 / A])

def B(A, q):
  return as_vector([0, k_r * q / A])

def S(A, q):
  return as_vector([0, k_r * q / A])

def dS_dU(A, q):
  return as_tensor([[0, 0], [- k_r * q / A ** 2, k_r / A]])

In [104]:
# Modify in order to pass only the singolar value of each expression at the inlet or at the outlet
def U(A, q):
  return np.array([A, q])

def H_vec(A, q):
  return np.array([[0, 1], [1190476.19048 * A ** 0.5 - (q / A) ** 2, 2 * q / A]])

def B_vec(A, q):
  return np.array([0, k_r * q / A])

def S_vec(A, q):
  return np.array([0, k_r * q / A])

def c_alpha(A, q, alpha = 1):
  return (1190476.19048 * A ** 0.5 + (q / A) ** 2 * alpha * (alpha - 1)) ** 0.5

def l1(A, q, alpha = 1):
  return np.array([c_alpha(A, q, alpha) - alpha * q / A, 1.])

def l2(A, q, alpha = 1):
  return np.array([- c_alpha(A, q, alpha) - alpha * q / A, 1.])

def CC(A, q, u_der, x):
  du_dz = project(u_der, V_der)(x)
  return U(A, q) - dt * H_vec(A, q) @ du_dz - dt * B_vec(A, q) + dt * f(x)

In [ ]:
def inlet_bc(A, q, u_der):
  q_inlet = (np.dot(l2(A(x_left), q(x_left)), CC(A(x_left), q(x_left), u_der, x_left)) - l2(A(x_left), q(x_left))[0] * U(A(x_left), q(x_left))[0] ) / l2(A(x_left), q(x_left))[1]
  return q_inlet

In [ ]:
def outlet_bc(A, q, u_der):
  matrix = np.array([[1, -1], [c_alpha(A(x_right), q(x_right), alpha) + alpha * q(x_right) / A(x_right), c_alpha(A(x_right), q(x_right), alpha) - alpha * q(x_right) / A(x_right)]])
  array = np.array([np.dot(l1(A(x_right), q(x_right)), CC(A(x_right), q(x_right), u_der, x_right)), np.dot(l2(A(x_right), q(x_right)), U(A(x_right), q(x_right)) - dt * S_vec(A(x_right), q(x_right)))])

  A_out, q_out = matrix @ array / (2 * c_alpha(A(x_right), q(x_right)))

  return A_out, q_out

In [ ]:
# Define the source term
f = Expression(('0','5.98399 * pow((0.1 * sin(6.28319*x[0]) * cos(6.28319*t) + 1), 0.5) * cos(6.28319*t) * cos(6.28319*x[0]) - 0.00025 * cos(6.28319*t) * cos(6.28319*x[0])'), degree = 2, t = 0)
df_dt = Expression(('0', '-1.87993 * sin(6.28319*t) * sin(6.28319*x[0]) * cos(6.28319*t) * cos(6.28319*x[0]) / pow((0.1 * sin(6.28319*x[0]) * cos(6.28319*t) + 1), 0.5) - 37.5985 * pow((0.1*sin(6.28319 * x[0]) * cos(6.28319*t) + 1), 0.5) * sin(6.28319*t) * cos(6.28319*x[0]) + 0.00157 * sin(6.28319*t) * cos(6.28319*x[0])'), degree = 2, t = 0)

f_n = interpolate(f, V)
df_dt_n = interpolate(df_dt, V)

In [105]:
# Define initial guess
A0 = 4e-4
q0 = 0
uh_old = interpolate(Expression(('A_tilde + a_tilde * sin(2 * pi / L * x[0])', 'q0'), degree = 2, A_tilde = A_tilde, a_tilde = a_tilde, L = L, q0 = q0), V)
Ah_old, qh_old = uh_old.split()

In [ ]:
# Boundary condition
inlet = 'near(x[0], 0)'
outlet = 'near(x[0], 1)'
A_inlet = A0
q_inlet = inlet_bc(Ah_old, qh_old, uh_old.dx(0))
A_outlet, q_outlet = outlet_bc(Ah_old, qh_old, uh_old.dx(0))
print(f'A inlet = {A_inlet}, q inlet = {q_inlet}')
print(f'A outlet = {A_outlet}, q outlet = {q_outlet}')

# Dirichlet bc
bc_A_inlet = DirichletBC(V.sub(0), A_inlet, inlet)
# I should impose compatibility condition
bc_q_inlet = DirichletBC(V.sub(1), q_inlet, inlet)

# Non reflecting bc
bc_A_outlet = DirichletBC(V.sub(0), A_outlet, outlet)
# I should impose compatibility condition
bc_q_outlet = DirichletBC(V.sub(1), q_outlet, outlet)

bc = [bc_A_inlet, bc_q_inlet, bc_A_outlet, bc_q_outlet]

A inlet = 0.0004, q inlet = 7.715704324885025e-05
A outlet = 0.00040025001830508667, q outlet = 3.8578662026772204e-05


In [ ]:
# Define trial functions and test functions
A, q = TrialFunctions(V)
v, z = TestFunctions(V)

# Define the variational problem
def LinearProblem(A_old, q_old):
  a = inner(A, v) * dx + inner(q, z) * dx

  L = (
        A_old * v * dx
      + q_old * z * dx
      + dt * ((F(A_old, q_old) - dt / 2 * dot(H(A_old, q_old), S(A_old, q_old))))[0] * v.dx(0) * dx
      + dt * ((F(A_old, q_old) - dt / 2 * dot(H(A_old, q_old), S(A_old, q_old))))[1] * z.dx(0) * dx
      + dt ** 2 / 2 * (dot(dS_dU(A_old, q_old), F(A_old, q_old).dx(0)))[0] * v * dx
      + dt ** 2 / 2 * (dot(dS_dU(A_old, q_old), F(A_old, q_old).dx(0)))[1] * z * dx
      - dt ** 2 / 2 * (dot(H(A_old, q_old), F(A_old, q_old).dx(0)))[0] * v.dx(0) * dx
      - dt ** 2 / 2 * (dot(H(A_old, q_old), F(A_old, q_old).dx(0)))[1] * z.dx(0) * dx
      - dt * (S(A_old, q_old) - dt / 2 * dot(dS_dU(A_old, q_old), S(A_old, q_old)))[0] * v * dx
      - dt * (S(A_old, q_old) - dt / 2 * dot(dS_dU(A_old, q_old), S(A_old, q_old)))[1] * z * dx
      + dt * f[0] * v * dx
      + dt * f[1] * z * dx
      + dt ** 2 / 2 * (dot(H(A_old, q_old), f_n))[0] * v.dx(0) * dx
      + dt ** 2 / 2 * (dot(H(A_old, q_old), f_n))[1] * z.dx(0) * dx
      + dt ** 2 / 2 * (- dot(dS_dU(A_old, q_old), f_n) + df_dt_n)[0] * v * dx
      + dt ** 2 / 2 * (- dot(dS_dU(A_old, q_old), f_n) + df_dt_n)[1] * z * dx
  )

  return a, L

In [ ]:
# Time stepping
uh = Function(V)
t = 0
i = 0
for n in range(round(num_steps)):

  i +=1

  # Update time step
  t += dt
  f.t = t
  df_dt.t = t
  A_exact.t = t
  q_exact.t = t

  # Solve the problem
  a, L = LinearProblem(Ah_old, qh_old)
  solve(a == L, uh, bc)
  Ah, qh = uh.split(deepcopy = True)

  # Compute errors
  if (i % 100 == 0):
    print(f'Interation {i} / {round(num_steps)}')
    A_e = interpolate(A_exact, V.sub(0).collapse())
    q_e = interpolate(q_exact, V.sub(1).collapse())
    error = np.array([errornorm(Ah, A_e, 'L2'), errornorm(qh, q_e, 'L2')])
    #error = np.array([(A.compute_vertex_values() - A_e.compute_vertex_values(mesh)).max(), (q.compute_vertex_values() - q_e.compute_vertex_values(mesh)).max()])
    print(f't = %.4f: error = {error}' % (t))
    print()

  # Update previous solution
  Ah_old.assign(Ah)
  qh_old.assign(qh)

  q_inlet = inlet_bc(Ah_old, qh_old, uh.dx(0))
  A_outlet, q_outlet = outlet_bc(Ah_old, qh_old, uh.dx(0))

Interation 100 / 5000
Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_d21520104177ceeafe086dfdeeef0fc023aaeb0c

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00734544 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.0188811 seconds.

INFO:FFC:Compiler stage 3: Optimizing intermediate representation
INFO:FFC:---------------------------

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling form ffc_form_8587d2bb5fb56b67bd9b3fcaaba6ae98d38da6bc

INFO:FFC:Compiler stage 1: Analyzing form(s)
INFO:FFC:-----------------------------------
DEBUG:FFC:  Preprocessing form using 'uflacs' representation family.
INFO:FFC:  
INFO:FFC:  Geometric dimension:       1
  Number of cell subdomains: 0
  Rank:                      0
  Arguments:                 '()'
  Number of coefficients:    1
  Coefficients:              '[f_6062]'
  Unique elements:           'DG4(?,?), Vector<1 x CG1(?,?)>'
  Unique sub elements:       'DG4(?,?), Vector<1 x CG1(?,?)>, CG1(?,?)'
  
INFO:FFC:  representation:    auto --> uflacs
INFO:FFC:  quadrature_rule:   auto --> default
INFO:FFC:  quadrature_degree: auto --> 8
INFO:FFC:  quadrature_degree: 8
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.0372736 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:---------------------

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_e1d7c5f86f11b91ad88226f568b8fa6c0d9d6bea

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00362372 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.0155256 seconds.

INFO:FFC:Compiler stage 3: Optimizing intermediate representati

t = 0.0200: error = [9.41454055e-07 2.74133876e-05]

Interation 200 / 5000
t = 0.0400: error = [9.31928758e-07 2.97728959e-05]

Interation 300 / 5000
t = 0.0600: error = [9.13047746e-07 5.27932790e-05]

Interation 400 / 5000
t = 0.0800: error = [8.85163618e-07 9.29259984e-05]

Interation 500 / 5000
t = 0.1000: error = [8.48794687e-07 1.45421404e-04]

Interation 600 / 5000
t = 0.1200: error = [8.04613867e-07 2.08219953e-04]

Interation 700 / 5000
t = 0.1400: error = [7.53440639e-07 2.79921673e-04]

Interation 800 / 5000
t = 0.1600: error = [6.96239250e-07 3.59227673e-04]

Interation 900 / 5000
t = 0.1800: error = [6.34128730e-07 4.44806456e-04]

Interation 1000 / 5000
t = 0.2000: error = [5.68413545e-07 5.35264960e-04]

Interation 1100 / 5000
t = 0.2200: error = [5.00650460e-07 6.29151323e-04]

Interation 1200 / 5000
t = 0.2400: error = [4.32780575e-07 7.24969339e-04]

Interation 1300 / 5000
t = 0.2600: error = [3.67378432e-07 8.21197910e-04]

Interation 1400 / 5000
t = 0.2800: error = 